In [114]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

import random
import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm # Colormaps
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import pandas as pd

sns.set_style('darkgrid')
np.random.seed(42)



In [115]:
class DataSets:
    """
    munClasses->specifies the number of classes the grid should have
    minRange->gives the lower limit of the classes and the points should be in
    maxRange->gives the upper limit of the classes and the points should be in
    """
    def __init__(myobject, numClasses, numpoints, minRange, maxRange):
        myobject.numClasses = numClasses
        myobject.numpoints = numpoints
        myobject.minRange = minRange
        myobject.maxRange = maxRange

In [116]:
    def multidimential_gauss(x, d, mean, covariance):
        """pdf of the multivariate normal distribution."""
        x_m = x - mean
        return (1. / (np.sqrt((2 * np.pi)**d * np.linalg.det(covariance))) * 
                np.exp(-(np.linalg.solve(covariance, x_m).T.dot(x_m)) / 2))

In [117]:
    def generate_surface(mean, covariance, d):
        """Helper function to generate density surface."""
        nb_of_x = 50 # grid size
        x1s = np.linspace(-10, 10, num=nb_of_x)
        x2s = np.linspace(-10, 10, num=nb_of_x)
        x1, x2 = np.meshgrid(x1s, x2s) # Generate grid
        pdf = np.zeros((nb_of_x, nb_of_x))
        # Fill the cost matrix for each combination of weights
        for i in range(nb_of_x):
            for j in range(nb_of_x):
                pdf[i,j] = multidimential_gauss(
                    np.matrix([[x1[i,j]], [x2[i,j]]]), 
                    d, mean, covariance)
        return x1, x2, pdf  

In [118]:
    def creating_distributions(mean1, mean2, covar, d,count):
        """makes things pretty - create illustrations of multiple gaussian distributions
        TODO- Change the graph size"""
        bidimentional_mean = np.matrix([[mean1], [mean2]])  # Mean
        bidimentional_covariance = np.matrix([
        [covar, 0.], 
        [0., covar]])  # Covariance
        x1, x2, p = generate_surface(
            bidimentional_mean, bidimentional_covariance, d)

        fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(20,10))
        con = axs.contourf(x1, x2, p, 33, cmap=cm.YlGnBu)
        axs.set_xlabel('$x_1$', fontsize=10)
        axs.set_ylabel('$x_2$', fontsize=10)
        axs.axis([-50, 50, -50, 50])
        axs.set_aspect('equal')
        axs.set_title('Distribution', fontsize=13)

        # Add colorbar and title
        fig.subplots_adjust(right=0.8)
        cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
        cbar = fig.colorbar(con, cax=cbar_ax)
        cbar.ax.set_ylabel('$p(x_1, x_2)$', fontsize=10)
        #plt.show()
        return fig


In [132]:
    def setting_mean_covariance():
        """generates multiple gaussian distributions/classes"""
        d=2
        with open ('GaussClasses.csv', mode='w') as Random_Gauss_Distributions:
            fieldnames = ['Mean1','Mean2', 'Covariance']
            writer=csv.DictWriter(Random_Gauss_Distributions,fieldnames=fieldnames)
            writer.writeheader()

            for i in range (numClasses):
                temp1 = random.gauss(minRange, maxRange) #First mean
                str_temp1=str(temp1)

                temp2 = random.gauss(minRange, maxRange) #second mean
                str_temp2=str(temp2)

                temp3 = random.gauss(MinRange, maxRange) #covariance
                str_temp3=str(temp2)

                writer.writerow({'Mean1':str_temp1,'Mean2':str_temp2,'Covariance':str_temp3})

In [133]:
    def read_distributions():
        """
        Reads all the generted Gaussian distributions and 
        calls the creating_distributions function tto make illustrations for each distribution 
        """
        d=2
        with open ('GaussClasses.csv', mode='r') as Random_Gauss_Distributions:
            csv_reader=csv.DictReader(Random_Gauss_Distributions)

            linecount=0
            pp = PdfPages('Save multiple plots as PDF.pdf')

            for row in csv_reader:

                if linecount == 0:
                    linecount+=1
                #elif linecount == 1:
                   # print(row['Mean1'])
                    #linecount+=1
                else:
                    #print(row['Mean1'])
                    print(linecount)

                    figure = creating_distributions(float(row['Mean1']), float(row['Mean2']),float(row['Covariance']), d,line_count)
                    pp.savefig(figure)
                    linecount+=1

In [134]:
    #distance of each class with the other
    def euclidean_distance(a,b):
        dist = np.linalg.norm(a - b)
        return dist

In [135]:
    #Creating a csv file
    def datasetOfPoints():
        """Creates random popints that ned to be classified into classes"""
        with open('PointsToBeClassified.csv', mode='w') as dist_write:
            feildnames = ['X','Y']
            writer = csv.DictWriter(dist_write, fieldnames = feildnames)
            writer.writeheader()
            for i in range (1,501):
                writer.writerow({'X':random.uniform(minRage, maxRange), 'Y':random.uniform(minRange, maxRange)})

        #find centers of each class
        lists=[]
        with open ('DataSet.csv', mode='r') as data_read:
            reader = csv.DictReader(data_read)
            list_count = 0
            for row in reader:
                if list_count == 0:
                    list_count+=1
                else:
                    lists.append(return_center(float(row['Mean1']), float(row['Mean2']), float(row['Covariance'])))
                    list_count+=1

In [136]:

    #x = float(input("Enter the x: "))
   # y = float(input("Enter the y: "))

    #setting_mean_covariance()
    #creating_distributions(0,0,2.2,2)
    #generate_surface([[0.],[0.]],[[2.2,0.],[0,2.2]],2)
    #read_dis()
    
def main():
    #creating all the datasets
        obj = DataSets(50, 10, 2.5, 100.95)
        obj.setting_mean_covariance()
        obj.read_distributions()
        obj.datasetOfPoints()
    

if __name__ == "__main__":
            main()


   
    

AttributeError: 'DataSets' object has no attribute 'setting_mean_covariance'